In [ ]:
git clone https://github.com/ultralytics/yolov5
pip install Pillow==8.3.2
pip install -qr yolov5/requirements.txt


In [34]:
file_path = 'objectDetection/smoke-detection/data.yaml'

try:
    with open(file_path, 'r') as file:
        file_contents = file.read()
        print(file_contents)
except FileNotFoundError:
    print(f"File '{file_path}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")


train: ../train/images
val: ../valid/images

nc: 1
names: ['smoke']


In [35]:
import yaml

data_yaml_path = 'objectDetection/smoke-detection/data.yaml'

with open(data_yaml_path, 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

print(f"Number of classes: {num_classes}")


Number of classes: 1


In [36]:
import yaml

# Define the path to your yolov5s.yaml file
yolov5s_yaml_path = "objectDetection/smoke-detection/yolov5/models/yolov5s.yaml"

# Read and print the contents of the yolov5s.yaml file
with open(yolov5s_yaml_path, 'r') as stream:
    yaml_contents = yaml.safe_load(stream)

print(yaml_contents)


{'nc': 80, 'depth_multiple': 0.33, 'width_multiple': 0.5, 'anchors': [[10, 13, 16, 30, 33, 23], [30, 61, 62, 45, 59, 119], [116, 90, 156, 198, 373, 326]], 'backbone': [[-1, 1, 'Conv', [64, 6, 2, 2]], [-1, 1, 'Conv', [128, 3, 2]], [-1, 3, 'C3', [128]], [-1, 1, 'Conv', [256, 3, 2]], [-1, 6, 'C3', [256]], [-1, 1, 'Conv', [512, 3, 2]], [-1, 9, 'C3', [512]], [-1, 1, 'Conv', [1024, 3, 2]], [-1, 3, 'C3', [1024]], [-1, 1, 'SPPF', [1024, 5]]], 'head': [[-1, 1, 'Conv', [512, 1, 1]], [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']], [[-1, 6], 1, 'Concat', [1]], [-1, 3, 'C3', [512, False]], [-1, 1, 'Conv', [256, 1, 1]], [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']], [[-1, 4], 1, 'Concat', [1]], [-1, 3, 'C3', [256, False]], [-1, 1, 'Conv', [256, 3, 2]], [[-1, 14], 1, 'Concat', [1]], [-1, 3, 'C3', [512, False]], [-1, 1, 'Conv', [512, 3, 2]], [[-1, 10], 1, 'Concat', [1]], [-1, 3, 'C3', [1024, False]], [[17, 20, 23], 1, 'Detect', ['nc', 'anchors']]]}


In [37]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))



In [38]:
%%writetemplate objectDetection/smoke-detection/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]


In [39]:
import os

# Define the path to the model file
model_file = 'objectDetection/smoke-detection/yolov5/runs/train/yolov5s_results/weights/best.pt'

# Check if the model file exists
if os.path.isfile(model_file):
    print(f"Model file '{model_file}' already exists. Skipping training.")
else:
    # Run the training command
    
    !python objectDetection/smoke-detection/yolov5/train.py --img 416 --batch 16 --epochs 100 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results --cache


Model file 'objectDetection/smoke-detection/yolov5/runs/train/yolov5s_results/weights/best.pt' already exists. Skipping training.


In [40]:
!python objectDetection/smoke-detection/yolov5/detect.py --weights objectDetection/smoke-detection/yolov5/runs/train/yolov5s_results/weights/best.pt --source Tests/smoke.mp4 --conf 0.4

detect: weights=['objectDetection/smoke-detection/yolov5/runs/train/yolov5s_results/weights/best.pt'], source=Tests/smoke.mp4, data=objectDetection\smoke-detection\yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=objectDetection\smoke-detection\yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to 'E:\New': No such file or directory
YOLOv5  2023-11-9 Python-3.11.7 torch-2.1.0+cpu CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7246518 parameters, 0 gradients
video 1/1 (1/451) E:\New folder\Tests\smoke.mp4: 384x640 1 smoke, 553.3ms
video 1/1 (2/451) E:\New folder\Tests\smoke.mp4: 384x640 2 smokes, 470.0ms
video 1/1 (3/451) E:\New folder\Tests

In [43]:
#fire-detection

import ultralytics
from ultralytics import YOLO
from IPython.display import Image

In [45]:
import os

# Check if the model file yolov8s.pt is present in the current directory
model_file = 'objectDetection/fire-detection/best.pt'
data_path = 'objectDetection/fire-detection/continuous_fire-6/data.yaml'
if os.path.isfile(model_file):
    print(f"Model file '{model_file}' already exists. Skipping training.")
else:
    # Run the training command if the model file is not present
    !yolo task=detect mode=train model=yolov8s.pt data={data_path} epochs=50 imgsz=640 plots=True


Model file 'objectDetection/fire-detection/best.pt' already exists. Skipping training.


In [46]:
model=YOLO("objectDetection/fire-detection/best.pt")
model.predict(source='Tests/1.jpeg', imgsz=640, conf=0.6, save= True)


image 1/1 e:\New folder\Tests\1.jpeg: 640x576 1 fire, 1064.4ms
Speed: 88.2ms preprocess, 1064.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 576)
Results saved to runs\detect\predict2


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'fire'}
 orig_img: array([[[ 68, 147, 220],
         [ 68, 147, 220],
         [ 68, 146, 222],
         ...,
         [ 24,  16,  26],
         [ 24,  16,  26],
         [ 24,  16,  26]],
 
        [[ 68, 147, 220],
         [ 68, 147, 220],
         [ 68, 146, 222],
         ...,
         [ 24,  16,  26],
         [ 24,  16,  26],
         [ 24,  16,  26]],
 
        [[ 70, 147, 220],
         [ 70, 147, 220],
         [ 68, 146, 222],
         ...,
         [ 24,  16,  26],
         [ 24,  16,  26],
         [ 24,  16,  26]],
 
        ...,
 
        [[  2,   7,   8],
         [  2,   7,   8],
         [  2,   7,   8],
         ...,
         [  4,   6,   7],
         [  2,   4,   5],
         [  2,   4,   5]],
 
        [[  3,   8,   9],
         [  3,   8,   9],
         [  3,   8,   9],
         ...,
         [  3,   5,   6],
     

In [51]:
# Test on a video using 'predict' mode
!yolo task=detect mode=predict model='objectDetection/fire-detection/best.pt' source='Tests/fire.mp4' imgsz=640 conf=0.6 save_txt=False save=True project= 'objectDetection/fire-detection/runs/detect'


Ultralytics YOLOv8.0.219 🚀 Python-3.11.7 torch-2.1.0+cpu CPU (Intel Core(TM) i5-6300U 2.40GHz)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

video 1/1 (1/150) e:\New folder\Tests\fire.mp4: 384x640 1 fire, 637.3ms
video 1/1 (2/150) e:\New folder\Tests\fire.mp4: 384x640 1 fire, 582.7ms
video 1/1 (3/150) e:\New folder\Tests\fire.mp4: 384x640 1 fire, 439.6ms
video 1/1 (4/150) e:\New folder\Tests\fire.mp4: 384x640 1 fire, 455.9ms
video 1/1 (5/150) e:\New folder\Tests\fire.mp4: 384x640 1 fire, 399.6ms
video 1/1 (6/150) e:\New folder\Tests\fire.mp4: 384x640 1 fire, 380.8ms
video 1/1 (7/150) e:\New folder\Tests\fire.mp4: 384x640 1 fire, 564.2ms
video 1/1 (8/150) e:\New folder\Tests\fire.mp4: 384x640 1 fire, 413.5ms
video 1/1 (9/150) e:\New folder\Tests\fire.mp4: 384x640 2 fires, 425.1ms
video 1/1 (10/150) e:\New folder\Tests\fire.mp4: 384x640 2 fires, 414.4ms
video 1/1 (11/150) e:\New folder\Tests\fire.mp4: 384x640 2 fires, 379.5ms
video 1/1 (12/150) e:\New 

In [52]:
# fire and smoke detection
import os

model_path = "objectDetection/fire-smokeobject-detection/latest_ckpt.pth"
train_path = 'objectDetection/fire-smoke-object-detection\\YOLOX\\tools\\train.py'


# Check if the model file already exists
if os.path.exists(model_path):
    print("Model file already exists. Skipping training.")
else:
    
    !python fire-smokeobject-detection/YOLOX/tools/train.py -f fire-smokeobject-detection/YOLOX/exps/example/custom/yolox_s.py -d 1 -b 16 --fp16 -o -c {model_path}


Model file already exists. Skipping training.


In [53]:
MODEL_PATH = "objectDetection/fire-smokeobject-detection/latest_ckpt.pth"


In [ ]:
TEST_IMAGE_PATH = "Tests/myvid6.mp4"
!python objectDetection/fire-smokeobject-detection/YOLOX/tools/demo.py video -f objectDetection/fire-smokeobject-detection/YOLOX/exps/example/custom/yolox_s.py -c {MODEL_PATH} --path {TEST_IMAGE_PATH} --conf 0.25 --nms 0.45 --tsize 640 --save_result --device gpu
